In [1]:
import torch
from sklearn.datasets import fetch_california_housing
from src.mssp import MSSP

In [2]:
data = fetch_california_housing()
X = torch.tensor(data.data, dtype=torch.double)
y = torch.tensor(data.target, dtype=torch.double)

In [3]:
RANDOM_SEED = 42
if RANDOM_SEED is not None:
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)


In [4]:
i = torch.randperm(len(X))
i_train = i[:int(len(X)*0.8)]
i_valid = i[int(len(X)*0.8):int(len(X)*0.9)]
i_test = i[int(len(X)*0.9):]

In [5]:
X_train, y_train = X[i_train], y[i_train]
X_valid, y_valid = X[i_valid], y[i_valid]
X_test, y_test = X[i_test], y[i_test]

In [6]:
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

torch.Size([16512, 8]) torch.Size([16512])
torch.Size([2064, 8]) torch.Size([2064])
torch.Size([2064, 8]) torch.Size([2064])


In [7]:
model = MSSP(
    n_best=120, 
    loss_fn="mse", 
    random_seed=RANDOM_SEED, 
    epochs=2, 
    diversity_ratio=0.75, 
    pow_cross=True,
    # cv=5
)

In [8]:
# model.fit(X_train, y_train, X_valid, y_valid)
model.fit(X_train, y_train)

# 0.42431482672691345

loss (mse): 0.6163 epoch: 0 , time: 0.43s
loss (mse): 0.4759 epoch: 1 , time: 1.64s
Best loss: 0.4759081304073334 after training for 1 epochs


In [9]:
print(model.evaluate(X_test, y_test, top_k=32))
print(model.evaluate(X_test, y_test, top_k=16))
print(model.evaluate(X_test, y_test, top_k=8))
print(model.evaluate(X_test, y_test, top_k=4))
print(model.evaluate(X_test, y_test, top_k=1))

(tensor(0.4702, dtype=torch.float64), tensor(2.2326, dtype=torch.float64))
(tensor(0.4615, dtype=torch.float64), tensor(2.1912, dtype=torch.float64))
(tensor(0.4452, dtype=torch.float64), tensor(2.1938, dtype=torch.float64))
(tensor(0.4428, dtype=torch.float64), tensor(2.2221, dtype=torch.float64))
(tensor(0.4375, dtype=torch.float64), tensor(2.1924, dtype=torch.float64))


In [10]:
model.model[0].nodes

{tensor(7): <src.graph.PrimitiveNode at 0x258f3b72300>,
 tensor(41): <src.graph.PrimitiveNode at 0x258f3678170>,
 tensor(55): <src.graph.PrimitiveNode at 0x258f3b707d0>,
 tensor(56): <src.graph.PrimitiveNode at 0x258f3b70a70>,
 (0, tensor(0), 'lin'): <src.graph.Node at 0x258f3b52ba0>,
 (0, tensor(110), 'lin'): <src.graph.Node at 0x258f3b70bc0>,
 'head': <src.graph.Node at 0x258f3b722d0>}

In [13]:
model.model[0].nodes['head'].left_child.key

(0, tensor(0), 'lin')